# STIS DATA Preprocessing

In [3]:
# Essential Pkg
import fnmatch
import os
import numpy as np
from osgeo import gdal
#### Unzip folders
import zipfile

### STEP I : unzip STIS files for each period 

In [7]:
### STEP I : Fxn to unzip  files for each period 
def unzip(input_dir,output_dir):
    """
    @input_dir : contains STIS data (ex : 1 year data, each zip file contains images (for multiple band)
                                    for a month)
    @output_dir : output directory, where you want t unzip files
    """
    files = os.listdir(input_dir)
    for file in files:
        filePath=input_dir+'/'+file
        print(filePath)
        with zipfile.ZipFile(filePath, 'r') as zip_ref:
            zip_ref.extractall(output_dir)

In [8]:
input_dir = 'ziped_file_directory'
output_dir = 'unziped_files_directory' 
# usage
unzip(input_dir,output_dir)

### STEP II : rename files for needed bands

In [5]:
##### Rename file names for visible band (B2,B3,B4,B5), in order to make then custom and readable
### ex : from SENTINEL2A_20170126-105612-238_L2A_T31TEJ_D_V1-4_ATB_R1.tif to 20170126_B1.tif

In [11]:
### STEP II : rename files for needed bands
def rename_file(directory, bandlist):
    for root, dirnames, filenames in os.walk(directory):
        for i in bandlist:
            for filename1 in fnmatch.filter(filenames,i):
#                 print(filename1)
                old_file = os.path.join(root, filename1)
#                 new_fname = filename1.split("_")[0]+'-'+filename1.split("_")[-1]
                new_fname = filename1.split("_")[1].split('-')[0]+'-'+filename1.split("_")[-1]
                
                new_file =  os.path.join(root, new_fname)
#                 print(new_file)
                os.rename(old_file, new_file)

In [12]:
# B2 (bleue), B3 (verte), B4 (rouge), et B8 (proche infra-rouge).
bandlist = ['*_FRE_B2.tif', '*_FRE_B3.tif', '*_FRE_B4.tif', '*_FRE_B8.tif']
directory = 'unziped_files_directory'
# usage
rename_file(directory,bandlist)

In [13]:
# fxn to compute ndvi
def ndvi(red, nir):
    return ((nir.astype('float32') - red.astype('float32')) / (nir+red))

### STEP III - Generate NDVI bands

In [15]:
####STEP III - Generate NDVI band
def ndvi_generator(directory):
    dic = {}
    nir_band = ['*-B4.tif', '*-B8.tif']
    outpath = ''
    liste =[]
    for root, dirnames, filenames in os.walk(directory):
        for x in filenames:
            tmp = x.split('-')[0]
            dic[tmp] =  {}
            for i in nir_band:
                for filename in fnmatch.filter(filenames, i):        
                    bd = filename.split('-')[-1].split('.')[0]
                    dic[tmp][bd] = os.path.join(root,filename)# filename # []
    dic = {k: v for k, v in dic.items() if v}
#     print(dic, len(dic))
    
    for K in dic:
    #     print(dic[K]['B4'])
        redbname = dic[K]['B4']
#         print(redbname.shape
        red_band_data =gdal.Open(redbname)
        red_band_array = red_band_data.ReadAsArray().astype('float32')
        print(red_band_array.shape)
    #     print(red_band_array)

        nirfile = dic[K]['B8']
        nir_band_data = gdal.Open(nirfile)
        nir_band_array = nir_band_data.ReadAsArray().astype('float32')
        print(nir_band_array.shape)
#         print(nir_band_array)

        # compute ndvi value
        ndvi_val = ndvi(red_band_array,nir_band_array)
        ###
#         outfile = nirfile.split('.')[0] + '-ndvi.tif'
        outfile = nirfile.split('.')[0] + '_NDVI.tif'
        print(outfile)

        x_pixels = red_band_array.shape[0] # number of pixels in x
        y_pixels = red_band_array.shape[1] # number of pixels in y
        print(x_pixels,y_pixels,K)

        driver = gdal.GetDriverByName('GTiff')
        ndvi_data = driver.Create(outfile,x_pixels, y_pixels, 1,gdal.GDT_Float32)
#         x = ndvi_data.GetRasterBand(1)
#         print(x)
        ndvi_data.GetRasterBand(1).WriteArray(ndvi_val)

        geotrans=red_band_data.GetGeoTransform() #  input GeoTranform information
        proj=red_band_data.GetProjection() #  projection information from input file

        ndvi_data.SetGeoTransform(geotrans) 
        ndvi_data.SetProjection(proj)
        ndvi_data.FlushCache()
        ndvi_data=None    

In [2]:
directory = "unziped_files_directory"
# # usage
ndvi_generator(directory)

In [ ]:
##### NOTE : Faire attention aux bandes utilisées, pour ne pas avoir des erreurs ci-dessous :
########### error : ValueError: array larger than output file, or offset off edge : Check the 
               # that you are using the right bands, if you accountred that error

### STEP IV - Generate multi-bande images

In [18]:
#### STEP IV - Generate multi-bande images
def multi_band_generator(data):            
    dic = {}
    nir_band = ['*-B2.tif','*-B3.tif','*-B4.tif', '*-B8.tif','*-B8_NDVI.tif']
    outpath = ''
    liste =[]
    for root, dirnames, filenames in os.walk(directory):
        for x in filenames:
            tmp = x.split('-')[0]
            dic[tmp] =  {}
            for i in nir_band:
                for filename in fnmatch.filter(filenames, i):        
                    bd = filename.split('-')[-1].split('.')[0]
                    dic[tmp][bd] = os.path.join(root,filename)# filename # []
    dic = {k: v for k, v in dic.items() if v}


    for K in dic:
        bluebname = dic[K]['B2']
        blue_band_data =gdal.Open(bluebname)
        blue_band_array = blue_band_data.ReadAsArray().astype('float32')

        greenbname = dic[K]['B3']
        green_band_data =gdal.Open(greenbname)
        green_band_array = green_band_data.ReadAsArray().astype('float32')


        redbname = dic[K]['B4']
        red_band_data =gdal.Open(redbname)
        red_band_array = red_band_data.ReadAsArray().astype('float32')

        nirbname = dic[K]['B8']
        nir_band_data =gdal.Open(nirbname)
        nir_band_array = nir_band_data.ReadAsArray().astype('float32')

        ndvibname = dic[K]['B8_NDVI']
        ndvi_band_data =gdal.Open(ndvibname)
        ndvi_band_array = ndvi_band_data.ReadAsArray().astype('float32')

        ###################
        # Create output filename based on input name 
    #     outfile = os.path.join(outpath,K.split('-')[0] + '_MULTI.tif')
    #     print(outfile)
#         outfile = './'+ K.split('.')[0] + '_MULTI.tif'
        
        r_path = '/unziped_files/multi_bands_tif' # specify outpath
        outfile = r_path +'/'+K+'_multi.tif' # name for each multiband file
        
        x_pixels = red_band_array.shape[0] # number of pixels in x
        y_pixels = red_band_array.shape[1] # number of pixels in y
        print(x_pixels,y_pixels)

        # Set up output GeoTIFF
        driver = gdal.GetDriverByName('GTiff')

        # Create driver using output filename, x and y pixels, # of bands, and datatype
        multibnd_data = driver.Create(outfile,x_pixels, y_pixels, 5,gdal.GDT_Float32)
        #multibnd_data1 = driver.Create(outfile1,x_pixels, y_pixels, 5,gdal.GDT_Float32)

        # 
        multibnd_data.GetRasterBand(1).WriteArray(blue_band_array)
        multibnd_data.GetRasterBand(2).WriteArray(green_band_array)
        multibnd_data.GetRasterBand(3).WriteArray(red_band_array)
        multibnd_data.GetRasterBand(4).WriteArray(nir_band_array)
        multibnd_data.GetRasterBand(5).WriteArray(ndvi_band_array)


        # Setting up the coordinate reference system of the output GeoTIFF
        geotrans = red_band_data.GetGeoTransform() #  input GeoTranform information
        proj = red_band_data.GetProjection() #  projection information from input file


        # GeoTransform parameters and projection on the output file
        multibnd_data.SetGeoTransform(geotrans) 
        multibnd_data.SetProjection(proj)
        multibnd_data.FlushCache()
        multibnd_data=None


In [3]:
directory = "unziped_files"
# usage
multi_band_generator(directory)

### STEP V -> Cut origin images with the study area shape
#### run in wrap_cut .sh file using gdalwarp
* dossier -> multi_bands folder
* target_shape_file.shp -> hape for the study area

### STEP VI -> Check that all images are valid. If not, only keep the good ones

### STEP VII -> Segementation
#### run in segmentation.sh file with otbcli_LargeScaleMeanShift
   * dossier -> multibands folder for a given year (stis for a year)

### STEP IIX -> Run Graph4SIT 
* to build graph for each STIS (ex : 2018)